In [1]:
#Preflop analyzer


In [2]:
#Goal create a Faise First In(RFI) table with position 
 #howto - need to parse my hands, then determine what position im in, and determine if im the RFI
 #howto - then, i need to matplotlib 6 or so plots from the individual positions
         #- i would like this to be in the standard (A-2 * A-2) plot coloring the played range 

#Goal 2 - take the data and report some sort of percentage of play for hand per position, otherwise we may misrepresent a super large range of hands 
 #howto - we need a boolean average of hand per position reported in our figure 
#Goal 

In [3]:
import pandas as pd

In [7]:
#ignitionHandHistoryFile = input("Text file dir:")
handhistoryfilename = input("file dir:")
handhistoryfilename.replace("\\","\\\\")
#handhistoryfilename.replace('','')                            
ignitionHands = open(handhistoryfilename,"r").read()




file dir:C:\programming\preflop2\IPHH\3.txt


In [8]:
testHand = ignitionHands.split("Ignition Hand")[6]
print(testHand)

 #3915272643 TBL#21036458 HOLDEM No Limit - 2020-04-04 16:43:10
Seat 1: Dealer ($10.81 in chips)
Seat 2: Small Blind ($9.46 in chips)
Seat 3: Big Blind ($10 in chips)
Seat 4: UTG ($9.75 in chips)
Seat 5: UTG+1 ($9.85 in chips)
Seat 6: UTG+2 ($19.66 in chips)
Seat 7: UTG+3 [ME] ($9.85 in chips)
Seat 8: UTG+4 ($9.45 in chips)
Seat 9: UTG+5 ($6.91 in chips)
Dealer : Set dealer [1] 
Small Blind : Small Blind $0.05 
Big Blind : Big blind $0.10 
*** HOLE CARDS ***
Dealer : Card dealt to a spot [Jd Ks] 
Small Blind : Card dealt to a spot [8s Kh] 
Big Blind : Card dealt to a spot [Ts 3s] 
UTG : Card dealt to a spot [8d 4c] 
UTG+1 : Card dealt to a spot [4h 3d] 
UTG+2 : Card dealt to a spot [6d 6h] 
UTG+3  [ME] : Card dealt to a spot [2d Jh] 
UTG+4 : Card dealt to a spot [2c Tc] 
UTG+5 : Card dealt to a spot [9s 7c] 
UTG : Folds
UTG+1 : Folds
UTG+2 : Raises $0.30 to $0.30
UTG+3  [ME] : Folds
UTG+4 : Folds
UTG+5 : Folds
Dealer : Calls $0.30 
Small Blind : Folds
Big Blind : Folds
*** FLOP *** [Jc

In [69]:
#completed functions 

def parseHandPos(preflopLine):
    #accept 1 line of preflop text 
    #returns hand and Position info
    
    if preflopLine.find("Card dealt to a spot") == -1: #this would mean this isnt a card assignemnt line #mistakes 
        positionInfo,leftcard,rightcard = None,None,None
    else: #text parse out 
        
        preflopLinewMeRemoved = preflopLine.replace(' [ME] ','')
        positionInfo = preflopLinewMeRemoved[:preflopLinewMeRemoved.find(":")-1]
        leftcard = preflopLinewMeRemoved[preflopLinewMeRemoved.find("[")+1:preflopLinewMeRemoved.find("[")+3]
        rightcard = preflopLinewMeRemoved[preflopLinewMeRemoved.find("[")+4:preflopLinewMeRemoved.find("[")+6]
    
    return positionInfo,leftcard,rightcard #cards have no fucntional order

def findSubstringInList(searchList, substring):
    #this useful fuction will return the index of the first list index that contains the substring
    
    if any(substring in string for string in searchList):
        return [idx for idx, s in enumerate(searchList) if substring in s][0] #returns the index location
    else:
        return -1 # returns -1 if the substring is not in the list

def preflopChunkParse(handHistory):
    ##assume that the last preflop aciton is the line before the "*** FLOP ***"
    handHistoryLines = handHistory.splitlines()
    
    preflopEndPhrase = "*** FLOP ***"
    preflopStartPhrase = "Card dealt to a spot"
    #using the phrase that starts the next phase
    preflopEnd =  findSubstringInList(handHistoryLines,preflopEndPhrase)
    
    #find the first list index in the reverse list that contains the phrase for card delt
    #this should be the last thing done before the preflop aciton 
    preflopStart = len(handHistoryLines) - findSubstringInList(handHistoryLines[::-1],preflopStartPhrase)
    
    #find the beginning and ending line
    preflopHandHistoryLines = handHistoryLines[preflopStart:preflopEnd]
    return preflopHandHistoryLines

def RFIAction(preflopHandHistoryLines):
    #expecting input of lines from the preflop
    
    RFIEndPhrase = "Raises"
    
    endingIndex = findSubstringInList(preflopHandHistoryLines, RFIEndPhrase)
    
    RFIActionLines = preflopHandHistoryLines[endingIndex]
    
    return RFIActionLines

def getAllLinesContainingSubString(listOfStrings, substring):
    
    return [s for s in listOfStrings if substring in s]



def spliceDeal(handHistory):
    handHistoryLines = handHistory.splitlines() 
    cardDealPhrase = "Card dealt to a spot"
    return getAllLinesContainingSubString(handHistoryLines, cardDealPhrase)


def createHoleCardDict(handHistory):
    
    #first reduce the Hand History to a list of relevant dealing card to position list
    dealtCardLines = spliceDeal(handHistory)
    
    #this line creats a dictionary by taking each line of the dealing action 
    #then it runs throught the parseHandpositions which give it the pos dict to the cards
    holeCardByPos = {parseHandPos(dealCardLine)[0]:(parseHandPos(dealCardLine)[1],parseHandPos(dealCardLine)[2]) for dealCardLine in dealtCardLines}
        
    return holeCardByPos


In [89]:
def getRFIPositionHand(agrigateHandHistory):
    #feed in Hand history output 
    #we want a dataframe that has Position and hand for every hand fed through a string
    
    #get line with RFI action 
    RFIAcitonLine = RFIAction(agrigateHandHistory.splitlines())
    
    preflopLinewMeRemoved = RFIAcitonLine.replace(' [ME] ','')
    RFIPosition = preflopLinewMeRemoved[:preflopLinewMeRemoved.find(":")-1]
    
    #use the line to get position and then feed into dictionary
    
    holeCardByPos = createHoleCardDict(agrigateHandHistory)
    
    hand = (holeCardByPos[RFIPosition])
    
    
    return RFIPosition, hand 
    
    
    

In [90]:
print(getRFIPositionHand(testHand))

('UTG+2', ('6d', '6h'))


In [92]:
print(createHoleCardDict(testHand))

{'Dealer': ('Jd', 'Ks'), 'Small Blind': ('8s', 'Kh'), 'Big Blind': ('Ts', '3s'), 'UTG': ('8d', '4c'), 'UTG+1': ('4h', '3d'), 'UTG+2': ('6d', '6h'), 'UTG+3': ('2d', 'Jh'), 'UTG+4': ('2c', 'Tc'), 'UTG+5': ('9s', '7c')}


In [104]:

#position_Headers = ['Dealer' , 'Small Blind' , 'Big Blind', 'UTG', 'UTG+1', 'UTG+2', 'UTG+3', 'UTG+4', 'UTG+5']

Header1 = 'Positions'
Header2 = 'Hand'

position_Headers = [Header1, Header2]
RFITable = pd.DataFrame(columns = position_Headers)



d = {Header1: getRFIPositionHand(testHand)[0], Header2 : (getRFIPositionHand(testHand)[1],)}
newRFIData = pd.DataFrame(d)


,Positions,Hand
0,UTG+2,"(6d, 6h)"


In [99]:
specialPhrase = "Card dealt to a spot"
#preflopChunkParse(testHand,specialPhrase)
print(createHoleCardDict(testHand)["Small Blind"])

('8s', 'Kh')
